# Building a Fraud Prediction Model with EvalML

In this demo, we will build an optimized fraud prediction model using EvalML. To optimize the pipeline, we will set up an objective function to minimize the percentage of total transaction value lost to fraud. At the end of this demo, we also show you how introducing the right objective during the training is over 4x better than using a generic machine learning metric like AUC.

In [1]:
%%time
import evalml
from evalml import AutoMLSearch
from evalml.objectives import FraudCost

CPU times: user 2.91 s, sys: 685 ms, total: 3.6 s
Wall time: 2 s


## Configure "Cost of Fraud" 

To optimize the pipelines toward the specific business needs of this model, we can set our own assumptions for the cost of fraud. These parameters are

* `retry_percentage` - what percentage of customers will retry a transaction if it is declined?
* `interchange_fee` - how much of each successful transaction do you collect?
* `fraud_payout_percentage` - the percentage of fraud will you be unable to collect
* `amount_col` - the column in the data the represents the transaction amount

Using these parameters, EvalML determines attempt to build a pipeline that will minimize the financial loss due to fraud.

In [2]:
%%time
fraud_objective = FraudCost(retry_percentage=.5,
                            interchange_fee=.02,
                            fraud_payout_percentage=.75,
                            amount_col='amount')

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 7.87 µs


## Search for best pipeline

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as the holdout set.

In [3]:
%%time
X, y = evalml.demos.load_fraud(n_rows=1000)

             Number of Features
Boolean                       1
Categorical                   6
Numeric                       5

Number of training examples: 1000
Targets
False    85.90%
True     14.10%
Name: fraud, dtype: object
CPU times: user 43 ms, sys: 4.18 ms, total: 47.2 ms
Wall time: 45.3 ms


EvalML natively supports one-hot encoding. Here we keep 1 out of the 6 categorical columns to decrease computation time.

In [4]:
%%time
cols_to_drop = ['datetime', 'expiration_date', 'country', 'region', 'provider']
for col in cols_to_drop:
    X.pop(col)

X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, y, problem_type='binary', test_size=0.2, random_state=0)

print(X.types)

                 Physical Type Logical Type Semantic Tag(s)
Data Column                                                
card_id                  Int64      Integer     ['numeric']
store_id                 Int64      Integer     ['numeric']
amount                   Int64      Integer     ['numeric']
currency              category  Categorical    ['category']
customer_present       boolean      Boolean              []
lat                    float64       Double     ['numeric']
lng                    float64       Double     ['numeric']
CPU times: user 12.2 ms, sys: 2.05 ms, total: 14.3 ms
Wall time: 12.6 ms


Because the fraud labels are binary, we will use `AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='binary')`. When we call `.search()`, the search for the best pipeline will begin. 

In [5]:
%%time
automl = AutoMLSearch(X_train=X_train, y_train=y_train,
                      problem_type='binary', 
                      objective=fraud_objective,
                      additional_objectives=['auc', 'f1', 'precision'],
                      max_batches=1,
                      optimize_thresholds=True)

automl.search()

Generating pipelines to search over...
*****************************
* Beginning pipeline search *
*****************************

Optimizing for Fraud Cost. 
Lower score is better.

Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: extra_trees, catboost, lightgbm, linear_model, xgboost, decision_tree, random_forest



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Batch 1: (1/9) Mode Baseline Binary Classification P... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Fraud Cost: 0.033
High coefficient of variation (cv >= 0.2) within cross validation scores. Mode Baseline Binary Classification Pipeline may not perform as estimated on unseen data.
Batch 1: (2/9) Logistic Regression Classifier w/ Imp... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Fraud Cost: 0.008
High coefficient of variation (cv >= 0.2) within cross validation scores. Logistic Regression Classifier w/ Imputer + One Hot Encoder + Standard Scaler may not perform as estimated on unseen data.
Batch 1: (3/9) Random Forest Classifier w/ Imputer +... Elapsed:00:03
	Starting cross validation
	Finished cross validation - mean Fraud Cost: 0.002
Batch 1: (4/9) XGBoost Classifier w/ Imputer + One H... Elapsed:00:04
	Starting cross validation
	Finished cross validation - mean Fraud Cost: 0.002
Batch 1: (5/9) CatBoost Classifier w/ Imput

### View rankings and select pipelines

Once the fitting process is done, we can see all of the pipelines that were searched, ranked by their score on the fraud detection objective we defined.

In [6]:
%%time
automl.rankings

CPU times: user 5.69 ms, sys: 1.01 ms, total: 6.69 ms
Wall time: 5.84 ms


,id,pipeline_name,score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,2,Random Forest Classifier w/ Imputer + One Hot ...,0.002066,0.002410,93.770164,False,{'Imputer': {'categorical_impute_strategy': 'm...
1,3,XGBoost Classifier w/ Imputer + One Hot Encoder,0.002066,0.002410,93.770164,False,{'Imputer': {'categorical_impute_strategy': 'm...
2,4,CatBoost Classifier w/ Imputer,0.002066,0.002410,93.770164,False,{'Imputer': {'categorical_impute_strategy': 'm...
3,5,Elastic Net Classifier w/ Imputer + One Hot En...,0.002066,0.002410,93.770164,False,{'Imputer': {'categorical_impute_strategy': 'm...
4,6,Extra Trees Classifier w/ Imputer + One Hot En...,0.002066,0.002410,93.770164,False,{'Imputer': {'categorical_impute_strategy': 'm...
5,1,Logistic Regression Classifier w/ Imputer + On...,0.008345,0.021246,74.838144,True,{'Imputer': {'categorical_impute_strategy': 'm...
6,8,Decision Tree Classifier w/ Imputer + One Hot ...,0.012008,0.003189,63.792696,True,{'Imputer': {'categorical_impute_strategy': 'm...
7,7,LightGBM Classifier w/ Imputer + One Hot Encoder,0.029568,0.021246,10.845280,True,{'Imputer': {'categorical_impute_strategy': 'm...
8,0,Mode Baseline Binary Classification Pipeline,0.033165,0.021246,0.000000,True,{'Baseline Classifier': {'strategy': 'mode'}}


To select the best pipeline we can call `automl.best_pipeline`.

In [7]:
%%time
best_pipeline = automl.best_pipeline

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 8.82 µs


### Describe pipelines

We can get more details about any pipeline created during the search process, including how it performed on other objective functions, by calling the `describe_pipeline` method and passing the `id` of the pipeline of interest.

In [8]:
%%time
automl.describe_pipeline(automl.rankings.iloc[1]["id"])

***************************************************
* XGBoost Classifier w/ Imputer + One Hot Encoder *
***************************************************

Problem Type: binary
Model Family: XGBoost

Pipeline Steps
1. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * categorical_fill_value : None
	 * numeric_fill_value : None
2. One Hot Encoder
	 * top_n : 10
	 * features_to_encode : None
	 * categories : None
	 * drop : None
	 * handle_unknown : ignore
	 * handle_missing : error
3. XGBoost Classifier
	 * eta : 0.1
	 * max_depth : 6
	 * min_child_weight : 1
	 * n_estimators : 100

Training
Training for binary problems.
Objective to optimize binary classification pipeline thresholds for: <evalml.objectives.fraud_cost.FraudCost object at 0x146dc4220>
Total training time (including CV): 1.4 seconds

Cross Validation
----------------
             Fraud Cost   AUC    F1  Precision # Training # Validation
0                 0.002 0.794 0.249      

## Evaluate on holdout data

Finally, since the best pipeline is already trained, we evaluate it on the holdout data.

Now, we can score the pipeline on the holdout data using both our fraud cost objective and the AUC (Area under the ROC Curve) objective.

In [9]:
%%time
best_pipeline.score(X_holdout, y_holdout, objectives=["auc", fraud_objective])

CPU times: user 128 ms, sys: 15.2 ms, total: 144 ms
Wall time: 134 ms


OrderedDict([('AUC', 0.8257890365448505),
             ('Fraud Cost', 0.0019819080665371956)])

## Why optimize for a problem-specific objective?

To demonstrate the importance of optimizing for the right objective, let's search for another pipeline using AUC, a common machine learning metric. After that, we will score the holdout data using the fraud cost objective to see how the best pipelines compare.

In [10]:
%%time
automl_auc = AutoMLSearch(X_train=X_train, y_train=y_train,
                          problem_type='binary',
                          objective='auc',
                          additional_objectives=['f1', 'precision'],
                          max_batches=1,
                          optimize_thresholds=True)

automl_auc.search()

Generating pipelines to search over...
*****************************
* Beginning pipeline search *
*****************************

Optimizing for AUC. 
Greater score is better.

Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: extra_trees, catboost, lightgbm, linear_model, xgboost, decision_tree, random_forest



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Batch 1: (1/9) Mode Baseline Binary Classification P... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean AUC: 0.500
Batch 1: (2/9) Logistic Regression Classifier w/ Imp... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean AUC: 0.758
Batch 1: (3/9) Random Forest Classifier w/ Imputer +... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean AUC: 0.844
Batch 1: (4/9) XGBoost Classifier w/ Imputer + One H... Elapsed:00:01
	Starting cross validation
	Finished cross validation - mean AUC: 0.851
Batch 1: (5/9) CatBoost Classifier w/ Imputer           Elapsed:00:02
	Starting cross validation
	Finished cross validation - mean AUC: 0.842
Batch 1: (6/9) Elastic Net Classifier w/ Imputer + O... Elapsed:00:03
	Starting cross validation
	Finished cross validation - mean AUC: 0.500
Batch 1: (7/9) Extra Trees Classifier w/ Imputer + O... Elapsed:00:03
	Starting cross validation
	Finished cross validation - mean AUC: 0.794
Batch 

Like before, we can look at the rankings of all of the pipelines searched and pick the best pipeline.

In [11]:
%%time
automl_auc.rankings

CPU times: user 59.8 ms, sys: 8.1 ms, total: 67.9 ms
Wall time: 4.9 ms


,id,pipeline_name,score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,7,LightGBM Classifier w/ Imputer + One Hot Encoder,0.852408,0.831763,70.481675,False,{'Imputer': {'categorical_impute_strategy': 'm...
1,3,XGBoost Classifier w/ Imputer + One Hot Encoder,0.850947,0.825328,70.189415,False,{'Imputer': {'categorical_impute_strategy': 'm...
2,2,Random Forest Classifier w/ Imputer + One Hot ...,0.844367,0.821765,68.873367,False,{'Imputer': {'categorical_impute_strategy': 'm...
3,4,CatBoost Classifier w/ Imputer,0.842214,0.825040,68.442897,False,{'Imputer': {'categorical_impute_strategy': 'm...
4,8,Decision Tree Classifier w/ Imputer + One Hot ...,0.825663,0.815100,65.132692,False,{'Imputer': {'categorical_impute_strategy': 'm...
5,6,Extra Trees Classifier w/ Imputer + One Hot En...,0.794342,0.741209,58.868418,False,{'Imputer': {'categorical_impute_strategy': 'm...
6,1,Logistic Regression Classifier w/ Imputer + On...,0.758291,0.673408,51.658208,False,{'Imputer': {'categorical_impute_strategy': 'm...
7,0,Mode Baseline Binary Classification Pipeline,0.500000,0.500000,0.000000,False,{'Baseline Classifier': {'strategy': 'mode'}}
8,5,Elastic Net Classifier w/ Imputer + One Hot En...,0.500000,0.500000,0.000000,False,{'Imputer': {'categorical_impute_strategy': 'm...


In [12]:
%%time
best_pipeline_auc = automl_auc.best_pipeline

CPU times: user 35 µs, sys: 6 µs, total: 41 µs
Wall time: 9.06 µs


In [13]:
%%time
# get the fraud score on holdout data
best_pipeline_auc.score(X_holdout, y_holdout,  objectives=["auc", fraud_objective])

CPU times: user 1.87 s, sys: 108 ms, total: 1.98 s
Wall time: 133 ms


OrderedDict([('AUC', 0.9067691029900331),
             ('Fraud Cost', 0.024416589442515867)])

In [14]:
%%time
# fraud score on fraud optimized again
best_pipeline.score(X_holdout, y_holdout, objectives=["auc", fraud_objective])

CPU times: user 2.14 s, sys: 103 ms, total: 2.25 s
Wall time: 147 ms


OrderedDict([('AUC', 0.8257890365448505),
             ('Fraud Cost', 0.0019819080665371956)])

When we optimize for AUC, we can see that the AUC score from this pipeline is better than the AUC score from the pipeline optimized for fraud cost. However, the losses due to fraud are over 3% of the total transaction amount when optimized for AUC and under 1% when optimized for fraud cost. As a result, we lose more than 2% of the total transaction amount by not optimizing for fraud cost specifically.

This happens because optimizing for AUC does not take into account the user-specified `retry_percentage`, `interchange_fee`, `fraud_payout_percentage` values. Thus, the best pipelines may produce the highest AUC but may not actually reduce the amount loss due to your specific type fraud.

This example highlights how performance in the real world can diverge greatly from machine learning metrics.